Adopted from https://www.kaggle.com/code/yxyyxy/rsna2024-training-baseline-2nd-stage/edit

# RSNA2024 LSDC Submission Baseline

This notebook will Let the model infer and make a submission.

### My other Notebooks
- [RSNA2024 LSDC Making Dataset](https://www.kaggle.com/code/itsuki9180/rsna2024-lsdc-making-dataset) 
- [RSNA2024 LSDC Training Baseline](https://www.kaggle.com/code/itsuki9180/rsna2024-lsdc-training-baseline) 
- [RSNA2024 LSDC Submission Baseline](https://www.kaggle.com/code/itsuki9180/rsna2024-lsdc-submission-baseline) <- you're reading now

# Import Libralies

In [1]:
!pip install segmentation_models_pytorch
!pip install natsort
!python /kaggle/input/deepspine-custom-dataset-preparation-process/main.py \
'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv' \
'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv' \
'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images' \
'/kaggle/input/2d-segmentation-of-sagittal-lumbar-spine-mri/simple_unet.pth'


  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 24.9 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=feadc6c0bc9556b75ab1d31df2e958c326d49a0d7f67f8bce0505da850270f5d
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=98097fa65cc5b9fe65b2c75f1d4976e5b9eb99b1a13d8bf5e088dd6d6b4d16ae
  Stored in directory: /root/.cache/pip/wheels/35/cb/

In [2]:
import os
import gc
import sys
from PIL import Image
import cv2
import math, random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

from collections import OrderedDict

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset, Sampler
from torch.optim import AdamW

import timm
from transformers import get_cosine_schedule_with_warmup

import albumentations as A


In [3]:
rd = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'

# Define the directory path
DATA_fromStage1 = '/kaggle/working'


# Config

In [4]:
OUTPUT_DIR = f'/kaggle/input/rsna2024-lsdc-training-baseline/rsna24-results'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
N_WORKERS = os.cpu_count()
USE_AMP = True
SEED = 1

IMG_SIZE = [256, 256]
N_LABELS = 5
N_CLASSES = 3 * N_LABELS

model_name_sag = 'efficientnet_b0'
model_name_axi = 'resnet34'
# resnet34
in_chans_sag = 15
in_chans_axi = 4

N_FOLDS = 5

BATCH_SIZE = 1

In [5]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [6]:
sample_sub = pd.read_csv(f'{rd}/sample_submission.csv')

In [7]:
LABELS = list(sample_sub.columns[1:])
LABELS

['normal_mild', 'moderate', 'severe']

In [8]:
CONDITIONS = [
    'spinal_canal_stenosis', 
    'left_neural_foraminal_narrowing', 
    'right_neural_foraminal_narrowing',
    'left_subarticular_stenosis',
    'right_subarticular_stenosis'
]

# Define the mapping for each level
level_mapping = {
    'L1/L2': 'l1_l2',
    'L2/L3': 'l2_l3',
    'L3/L4': 'l3_l4',
    'L4/L5': 'l4_l5',
    'L5/S1': 'l5_s1'
}


In [9]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [10]:
# Define the path to CSV file
csv_file_path = DATA_fromStage1 + '/sagittalT2/dataset_metadata.csv'
# Read the CSV file
dataset_metadata = pd.read_csv(csv_file_path)

# Print the number of columns and rows in the DataFrame
num_rows = dataset_metadata.shape[0]
print(dataset_metadata)

   Unnamed: 0                                          L4/L5  \
0    44036939  /kaggle/working/sagittalT2/44036939/L4_L5.npy   

                                           L3/L4  \
0  /kaggle/working/sagittalT2/44036939/L3_L4.npy   

                                           L2/L3  \
0  /kaggle/working/sagittalT2/44036939/L2_L3.npy   

                                           L1/L2  \
0  /kaggle/working/sagittalT2/44036939/L1_L2.npy   

                                           L5/S1  
0  /kaggle/working/sagittalT2/44036939/L5_S1.npy  


In [11]:
'''# one npy file example
print("--------------------- SagittalT2 samples -----------")
example_npy_fn = dataset_metadata['L4/L5'][0]
path_split = example_npy_fn.split('/')

path = os.path.join(DATA_fromStage1, path_split[3], path_split[4], path_split[5])
# Saggital T2 ------------  Load the .npy file
npy_sagT2_path = os.path.join(DATA_fromStage1 , path_split[3], path_split[4], path_split[5])
npy_sagT2 = np.load(npy_sagT2_path).astype(np.float32)

# Axial T2 ------------ Load the .npy file
npy_AxiT2_path = os.path.join(DATA_fromStage1 , "axialT2", path_split[4], path_split[5])
npy_AxiT2 = np.load(npy_AxiT2_path).astype(np.float32)

print("--------------------- SagittalT2 samples ----------")
# Print the structure of the data
print(f"Shape of the data: {npy_sagT2.shape}")
print(f"Data type: {npy_sagT2.dtype}")
# Show the 15 slices one by one
for i in range(15):
    plt.imshow(npy_sagT2[i], cmap='gray')
    plt.title(f'Slice {i+1}')
    plt.axis('off')
    plt.show()

print("--------------------- AxialT2 samples ----------")
print(f"Shape of the data: {npy_AxiT2.shape}")
print(f"Data type: {npy_AxiT2.dtype}")
for i in range(5):
    img = npy_AxiT2[i]
    plt.imshow(img, cmap='gray')
    plt.title(f'Slice {i+1}')
    plt.axis('off')
    plt.show()
'''

'# one npy file example\nprint("--------------------- SagittalT2 samples -----------")\nexample_npy_fn = dataset_metadata[\'L4/L5\'][0]\npath_split = example_npy_fn.split(\'/\')\n\npath = os.path.join(DATA_fromStage1, path_split[3], path_split[4], path_split[5])\n# Saggital T2 ------------  Load the .npy file\nnpy_sagT2_path = os.path.join(DATA_fromStage1 , path_split[3], path_split[4], path_split[5])\nnpy_sagT2 = np.load(npy_sagT2_path).astype(np.float32)\n\n# Axial T2 ------------ Load the .npy file\nnpy_AxiT2_path = os.path.join(DATA_fromStage1 , "axialT2", path_split[4], path_split[5])\nnpy_AxiT2 = np.load(npy_AxiT2_path).astype(np.float32)\n\nprint("--------------------- SagittalT2 samples ----------")\n# Print the structure of the data\nprint(f"Shape of the data: {npy_sagT2.shape}")\nprint(f"Data type: {npy_sagT2.dtype}")\n# Show the 15 slices one by one\nfor i in range(15):\n    plt.imshow(npy_sagT2[i], cmap=\'gray\')\n    plt.title(f\'Slice {i+1}\')\n    plt.axis(\'off\')\n    

# Define Dataset

In [12]:
class RSNA24TestDataset(Dataset):
    def __init__(self, df_fn, Slice_len_Sag=15, Slice_len_Axi=4, transform=None, trainsform_axis=None):
        self.df_fn = df_fn
        self.transform = transform
        self.trainsform_axis = trainsform_axis
        self.Slice_len_Sag = Slice_len_Sag
        self.Slice_len_Axi = Slice_len_Axi
        # Select all rows where the 'Name' column has the value 'Alice'
    def __len__(self):
        return len(self.df_fn)

    def __getitem__(self, idx):
        row_idx = self.df_fn.iloc[idx]
        # Extract the ID
        st_id = row_idx['Unnamed: 0']
        # Extract the column values and level names
        Data_idx = []
        for level in self.df_fn.columns[1:]:
            filename = row_idx[level]
            path_split = filename.split('/')
            # Saggital T2 ------------  Load the .npy file
            npy_sagT2_path = os.path.join(DATA_fromStage1 , path_split[3], path_split[4], path_split[5])
            npy_sagT2 = np.load(npy_sagT2_path).astype(np.float32)
            current_length = npy_sagT2.shape[0]
            if current_length > self.Slice_len_Sag:
                indices = np.linspace(0, current_length - 1, self.Slice_len_Sag, dtype=int)
                npy_sagT2 = npy_sagT2[indices, :, :]
            elif current_length < self.Slice_len_Sag:
                indices = np.linspace(0, current_length - 1, self.Slice_len_Sag, dtype=int)
                npy_sagT2 = npy_sagT2[indices, :, :]

            # Axial T2 ------------ Load the .npy file
            npy_AxiT2_path = os.path.join(DATA_fromStage1 , "axialT2", path_split[4], path_split[5])
            npy_AxiT2 = np.load(npy_AxiT2_path).astype(np.float32)
            current_length = npy_AxiT2.shape[0]
            if current_length > self.Slice_len_Axi:
                indices = np.linspace(0, current_length - 1, self.Slice_len_Axi, dtype=int)
                npy_AxiT2 = npy_AxiT2[indices, :, :]
            elif current_length < self.Slice_len_Axi:
                indices = np.linspace(0, current_length - 1, self.Slice_len_Axi, dtype=int)
                npy_AxiT2 = npy_AxiT2[indices, :, :]
            #  npy_sagT2 and npy_AxiT2 are in the format [channels, height, width]
            npy_sagT2 = np.transpose(npy_sagT2, (1, 2, 0))  # Transpose to [height, width, channels]
            npy_AxiT2 = np.transpose(npy_AxiT2, (1, 2, 0)) 
            if self.transform is not None:
                npy_sagT2 = self.transform(image=npy_sagT2)['image']
                npy_AxiT2 = self.trainsform_axis(image=npy_AxiT2)['image']
            # transpose back to the original format
            npy_sagT2 = np.transpose(npy_sagT2, (2, 0, 1))  # Transpose back to [channels, height, width]
            npy_AxiT2 = np.transpose(npy_AxiT2, (2, 0, 1)) 
            Data_idx.append((npy_sagT2, npy_AxiT2, level))
        
        return st_id, Data_idx


In [13]:
transforms_test_Sag = A.Compose([
    A.Resize(84, 160),
    A.Normalize(mean=0.5, std=0.5)
])

transforms_test = A.Compose([
    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.Normalize(mean=0.5, std=0.5)
])


In [14]:
test_ds = RSNA24TestDataset(dataset_metadata, transform=transforms_test_Sag, trainsform_axis=transforms_test)
test_dl = DataLoader(
    test_ds, 
    batch_size=1, 
    shuffle=False,
    num_workers=N_WORKERS,
    pin_memory=True,
    drop_last=False
)


In [15]:
'''from collections import Counter

sagT2_slicenum = []
AxiT2_slicenum = []

print(test_dl.__len__())

# Iterate through the data loader and append slice numbers
for i, (st_id, data) in enumerate(test_dl):
    for npy_sagT2, npy_AxiT2, level in data:
        sagT2_slicenum.append(npy_sagT2.shape)
        AxiT2_slicenum.append(npy_AxiT2.shape)
        assert not torch.isnan(npy_sagT2).any(), "NaN values found in npy_sagT2"
        assert not torch.isnan(npy_AxiT2).any(), "NaN values found in npy_AxiT2"
        print(level)

# Count the occurrences of each unique value in the lists
sagT2_counts = Counter(sagT2_slicenum)
AxiT2_counts = Counter(AxiT2_slicenum)

print("Occurrences of each unique value in sagT2_slicenum:")
for value, count in sagT2_counts.items():
    print(f"Value: {value}, Count: {count}")

print("Occurrences of each unique value in AxiT2_slicenum:")
for value, count in AxiT2_counts.items():
    print(f"Value: {value}, Count: {count}")'''


'from collections import Counter\n\nsagT2_slicenum = []\nAxiT2_slicenum = []\n\nprint(test_dl.__len__())\n\n# Iterate through the data loader and append slice numbers\nfor i, (st_id, data) in enumerate(test_dl):\n    for npy_sagT2, npy_AxiT2, level in data:\n        sagT2_slicenum.append(npy_sagT2.shape)\n        AxiT2_slicenum.append(npy_AxiT2.shape)\n        assert not torch.isnan(npy_sagT2).any(), "NaN values found in npy_sagT2"\n        assert not torch.isnan(npy_AxiT2).any(), "NaN values found in npy_AxiT2"\n        print(level)\n\n# Count the occurrences of each unique value in the lists\nsagT2_counts = Counter(sagT2_slicenum)\nAxiT2_counts = Counter(AxiT2_slicenum)\n\nprint("Occurrences of each unique value in sagT2_slicenum:")\nfor value, count in sagT2_counts.items():\n    print(f"Value: {value}, Count: {count}")\n\nprint("Occurrences of each unique value in AxiT2_slicenum:")\nfor value, count in AxiT2_counts.items():\n    print(f"Value: {value}, Count: {count}")'

# Define Model

In [16]:
class LevelHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LevelHead, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
class RSNA24Model_Hybrid(nn.Module):
    def __init__(self, model_name_sag, model_name_axi, in_chans_sag, in_chans_axi, num_classes, level_names):
        super(RSNA24Model_Hybrid, self).__init__()
        self.model_sag = timm.create_model(model_name_sag, in_chans=in_chans_sag, global_pool='avg'
                                           , pretrained=True, features_only=False)
        self.model_axi = timm.create_model(model_name_axi, in_chans=in_chans_axi, global_pool='avg'
                                           , pretrained=True, features_only=False)
        
        # Replace the last layer with an identity layer
        if hasattr(self.model_sag, 'classifier'):
            self.model_sag.classifier = nn.Identity()
        elif hasattr(self.model_sag, 'fc'):
            self.model_sag.fc = nn.Identity()
        
        if hasattr(self.model_axi, 'classifier'):
            self.model_axi.classifier = nn.Identity()
        elif hasattr(self.model_axi, 'fc'):
            self.model_axi.fc = nn.Identity()
        
        # Get the output feature sizes
        with torch.no_grad():
            sample_input_sag = torch.randn(1, in_chans_sag, 84, 160)
            sample_input_axi = torch.randn(1, in_chans_axi, 256, 256)
            output_sag = self.model_sag(sample_input_sag)
            output_axi = self.model_axi(sample_input_axi)
        
        # Define the final fully connected layers for each task
        self.fc_heads = nn.ModuleDict({
            level: LevelHead(output_sag.shape[1] + output_axi.shape[1], num_classes) for level in level_names
        })
        
    def forward(self, x_sag, x_axi, level):
        x_sag = self.model_sag(x_sag)
        x_axi = self.model_axi(x_axi)
        x = torch.cat((x_sag, x_axi), dim=1)
        x = self.fc_heads[level[0]](x) # the input level is a array associated with batch size
        return x


# Load Models

In [17]:
import glob
CKPT_PATHS = glob.glob('/kaggle/input/rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-*.pt')
CKPT_PATHS = sorted(CKPT_PATHS)

In [18]:
# Ensure device is set correctly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = []
for i, cp in enumerate(CKPT_PATHS):
    print(f'loading {cp}...')
    model = RSNA24Model_Hybrid(model_name_sag, model_name_axi,
                               in_chans_sag, in_chans_axi, 
                               num_classes=N_CLASSES, level_names=level_mapping.keys())
    model.load_state_dict(torch.load(cp, map_location=device))
    model.eval()
    model.to(device)
    models.append(model)
    

loading /kaggle/input/rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-0.pt...


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

loading /kaggle/input/rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-1.pt...
loading /kaggle/input/rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-2.pt...
loading /kaggle/input/rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-3.pt...
loading /kaggle/input/rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-4.pt...


# Inference loop

In [19]:
autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half)
y_preds = []
row_names = []

with tqdm(test_dl, leave=True) as pbar:
    with torch.no_grad():
        for i, (st_id, data) in enumerate(test_dl):
            for npy_sagT2, npy_AxiT2, level in data:
                pred_per_study = np.zeros((N_LABELS, 3))
                npy_sagT2 = npy_sagT2.to(device)
                npy_AxiT2 = npy_AxiT2.to(device)
                with torch.cuda.amp.autocast():    
                    for m in models:
                        y = m(npy_sagT2, npy_AxiT2, level)[0]
                        for col in range(N_LABELS):
                            pred = y[col*3:col*3+3]
                            y_pred = pred.float().softmax(0).cpu().numpy()
                            pred_per_study[col] += y_pred / len(models)
                    y_preds.append(pred_per_study)
                    
                for cond in CONDITIONS:
                    row_name = f"{str(st_id.item())}_{cond}_{level_mapping[''.join(level)]}"
                    row_names.append(row_name)

y_preds = np.concatenate(y_preds, axis=0)
                    
print(len(row_names))
print(len(y_preds))

  0%|          | 0/1 [00:01<?, ?it/s]

25
25


# Make Submission

In [20]:
sub = pd.DataFrame()
sub['row_id'] = row_names
sub[LABELS] = y_preds
sub.head(25)

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l4_l5,0.355254,0.250128,0.394618
1,44036939_left_neural_foraminal_narrowing_l4_l5,0.308700,0.413929,0.277371
2,44036939_right_neural_foraminal_narrowing_l4_l5,0.331669,0.413591,0.254739
3,44036939_left_subarticular_stenosis_l4_l5,0.228818,0.316435,0.454747
4,44036939_right_subarticular_stenosis_l4_l5,0.220210,0.330893,0.448898
5,44036939_spinal_canal_stenosis_l3_l4,0.403202,0.289737,0.307061
6,44036939_left_neural_foraminal_narrowing_l3_l4,0.399095,0.407205,0.193700
7,44036939_right_neural_foraminal_narrowing_l3_l4,0.402412,0.404273,0.193315
8,44036939_left_subarticular_stenosis_l3_l4,0.294630,0.343371,0.361998
9,44036939_right_subarticular_stenosis_l3_l4,0.304565,0.330558,0.364876


In [21]:
sub.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l4_l5,0.355254,0.250128,0.394618
1,44036939_left_neural_foraminal_narrowing_l4_l5,0.308700,0.413929,0.277371
2,44036939_right_neural_foraminal_narrowing_l4_l5,0.331669,0.413591,0.254739
3,44036939_left_subarticular_stenosis_l4_l5,0.228818,0.316435,0.454747
4,44036939_right_subarticular_stenosis_l4_l5,0.220210,0.330893,0.448898


# Conclusion
We created the dataset, performed training, and inference in this notebook. 

This competition is a bit complicated to handle the dataset, so there may be a better way.

I think there are many other areas to improve in my notebook. I hope you can learn from my notebook and get a better score.